<a href="https://colab.research.google.com/github/JayroMartinez/game_of_life_sim/blob/main/game_of_life_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
# CLONE OR UPDATE REPO

if [ ! -d game_of_life_sim ]; then
  git clone https://github.com/JayroMartinez/game_of_life_sim.git -q
else
  git -C game_of_life_sim pull -q
fi
pip install -q -r game_of_life_sim/requirements.txt

In [ ]:
# ENVIRONMENT SETUP
import os, sys
sys.path.append("/content/game_of_life_sim/src")
os.chdir("/content/game_of_life_sim")
print("Environment ready.")

In [ ]:
# SINGLE GIF SIMULATION UNTIL COLLAPSE (larger + inverted colors)
import imageio
import numpy as np
import zlib
from src.life_core import life_step
from IPython.display import Image, display

# parameters
size  = 60       # board side length
prob  = 0.25     # initial alive-cell probability
fps   = 10       # frames per second
scale = 5        # how much to upscale each cell

# initialize
board = np.random.rand(size, size) < prob
frames = []
seen = {zlib.crc32(board.tobytes())}
cycles = 0

# simulate until collapse
while True:
    # create 0–255 image, invert so living=0 (black), dead=255 (white)
    img = ((~board).astype(np.uint8)) * 255
    # upscale each pixel by `scale`
    big = np.kron(img, np.ones((scale, scale), dtype=np.uint8))
    frames.append(big)

    # next generation
    board = life_step(board)
    cycles += 1
    h = zlib.crc32(board.tobytes())
    if not board.any() or h in seen:
        # final frame
        img = ((~board).astype(np.uint8)) * 255
        big = np.kron(img, np.ones((scale, scale), dtype=np.uint8))
        frames.append(big)
        break
    seen.add(h)

print(f"Collapsed after {cycles} cycles")

# write and display GIF
gif_path = "simulation_until_collapse.gif"
imageio.mimsave(gif_path, frames, fps=fps)
display(Image(filename=gif_path))

In [ ]:
# PARALLEL SIMULATION
# --runs: Simulations to run --size: Board size length --prob: Initial alive-cell probability
!python run_sim.py --runs 100 --size 60 --prob 0.33